In [1]:
import pandas as pd
import umap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
import hdbscan

%matplotlib inline

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder


In [3]:
final_df = pd.read_csv("../data/processed/final_df.csv")

In [4]:
#find null values
final_df[final_df.isnull().any(axis=1)]

,id_student,age_band,gender,highest_education,n_assignments,mean_score,final_result,total_clicks,studied_credits,region,code_module,code_presentation,press_mod
797,186780,35-55,F,A Level or Equivalent,0,NaN,Withdrawn,30,120,North Western Region,BBB,2013B,2013B-BBB
1734,549713,35-55,F,Lower Than A Level,0,NaN,Withdrawn,75,60,West Midlands Region,BBB,2013B,2013B-BBB
1843,554393,35-55,M,A Level or Equivalent,0,NaN,Fail,21,60,West Midlands Region,BBB,2013B,2013B-BBB
4498,606501,0-35,F,A Level or Equivalent,0,NaN,Withdrawn,9,120,West Midlands Region,BBB,2014B,2014B-BBB
6171,654422,0-35,F,No Formal quals,0,NaN,Withdrawn,31,60,East Midlands Region,BBB,2014J,2014J-BBB
13730,555297,0-35,M,Lower Than A Level,0,NaN,Withdrawn,155,60,North Western Region,DDD,2013B,2013B-DDD
13768,557247,0-35,F,A Level or Equivalent,0,NaN,Withdrawn,576,60,Yorkshire Region,DDD,2013B,2013B-DDD
13769,557247,0-35,F,A Level or Equivalent,0,NaN,Withdrawn,576,60,Yorkshire Region,DDD,2013J,2013J-DDD
14355,427248,35-55,F,A Level or Equivalent,0,NaN,Withdrawn,317,120,North Western Region,DDD,2013J,2013J-DDD
17039,676642,0-35,M,Lower Than A Level,0,NaN,Withdrawn,62,60,South West Region,DDD,2014J,2014J-DDD


In [6]:
final_df.isnull().sum()

id_student            0
age_band              0
gender                0
highest_education     0
n_assignments         0
mean_score           19
final_result          0
total_clicks          0
studied_credits       0
region                0
code_module           0
code_presentation     0
press_mod             0
dtype: int64

In [7]:
final_df["mean_score"].fillna(72.83, inplace = True)

In [8]:
final_df.set_index(['id_student'], inplace=True)

In [9]:
def drop_col (df, list):
    df.drop(columns=(list), inplace=True)
    return df

In [10]:
col_drop = ["press_mod", "code_presentation",]

In [11]:
drop_col (final_df, col_drop)

,age_band,gender,highest_education,n_assignments,mean_score,final_result,total_clicks,studied_credits,region,code_module
id_student,,,,,,,,,,
11391,55<=,M,HE Qualification,5,82.000000,Pass,934,240,East Anglian Region,AAA
28400,35-55,F,HE Qualification,5,66.400000,Pass,1435,60,Scotland,AAA
31604,35-55,F,A Level or Equivalent,5,76.000000,Pass,2158,60,South East Region,AAA
32885,0-35,F,Lower Than A Level,5,54.400000,Pass,1034,60,West Midlands Region,AAA
38053,35-55,M,A Level or Equivalent,5,68.000000,Pass,2445,60,Wales,AAA
...,...,...,...,...,...,...,...,...,...,...
2620947,0-35,F,A Level or Equivalent,9,88.888889,Distinction,476,30,Scotland,GGG
2645731,35-55,F,Lower Than A Level,9,88.111111,Distinction,893,30,East Anglian Region,GGG
2648187,0-35,F,A Level or Equivalent,9,76.666667,Pass,312,30,South Region,GGG


In [20]:
clustering_df = pd.DataFrame(data=final_df, columns=["n_assignments","mean_score", "total_clicks","studied_credits" ])

In [21]:

FEATS = ['n_assignments', 'mean_score', 'total_clicks', 'studied_credits']

In [22]:
transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), 
                ('scaler', StandardScaler())])

In [23]:
preprocessor = \
ColumnTransformer(transformers=[('num', transformer, FEATS)])

In [24]:
preprocessor

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('num',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0)),
                                                 ('scaler',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                             

In [25]:
clustering_1 = pd.DataFrame(data=preprocessor.fit_transform(final_df))

# UMAP Standard Scaler

In [26]:
clustering_1 = clustering_1.to_numpy()

In [27]:
reducer_1 = umap.UMAP(random_state=42)
reducer_1.fit(clustering_1.data)

UMAP(a=None, angular_rp_forest=False, b=None,
     force_approximation_algorithm=False, init='spectral', learning_rate=1.0,
     local_connectivity=1.0, low_memory=False, metric='euclidean',
     metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
     n_neighbors=15, negative_sample_rate=5, output_metric='euclidean',
     output_metric_kwds=None, random_state=42, repulsion_strength=1.0,
     set_op_mix_ratio=1.0, spread=1.0, target_metric='categorical',
     target_metric_kwds=None, target_n_neighbors=-1, target_weight=0.5,
     transform_queue_size=4.0, transform_seed=42, unique=False, verbose=False)

In [ ]:
embedding_1 = reducer_1.transform(clustering_1.data)